# LangChain

<center>

![LangChain](https://www.unite.ai/wp-content/uploads/2023/08/Heisenbergforlife_A_vivid_scene_where_a_striking_green_parrot_a_0607a84e-c5b8-41c7-8c9f-f3d154a0c7c2.png)
</center>

## 05. Chains

#### [Reference Docs](https://python.langchain.com/v0.1/docs/modules/chains/)

Central to LangChain is vital-component known as **LangChain Chains**, forming the core connection among one or several large lanaguage models(LLMs). In certain sophisticated applications, it becomes necessary to chain LLMs together, either with each other or with other elements.

LangChain provides two high level frameworks for **"chaining"** components.
- __Chain Interface__

- __LangChain Expression Language(LCEL)__ 

In [1]:
# Import necessary libraries
import os 
import openai 
import langchain
from dotenv import load_dotenv
from langchain_openai import OpenAI

# Load environment variables form .env file
load_dotenv()

# Access environment variables and setup OpenAI client
openai_key = os.getenv("OPENAI_API_KEY")
openai.api_key = openai_key

# Initialize OpenAI LLM client
client = OpenAI(openai_api_key=openai_key)  

In [2]:
# Import PromptTemplate
from langchain.prompts import PromptTemplate

# Create PromptTemplate
prompt = PromptTemplate.from_template(
    input_vairable= ["product"],
    template = "Where can I buy {product}? Give the response in a list format."
    )

### 5.1 Types of Framework for Chaining Components

#### 5.1.1 Chain Interface

> __Deprecated__

In [3]:
# Import LangChain Chain
from langchain.chains import LLMChain

chain = LLMChain(prompt=prompt, llm=client)

/home/luluw/Desktop/LangChain-Generative-AI/env/lib/python3.11/site-packages/langchain_core/_api/deprecation.py:139: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 1.0. Use RunnableSequence, e.g., `prompt | llm` instead.
  warn_deprecated(


In [4]:
response = chain.invoke("Graphic Card Nvidia RTX 4090")
response

{'product': 'Graphic Card Nvidia RTX 4090',
 'text': "\n\n1. Nvidia Online Store - The official website of Nvidia offers various RTX 4090 models for purchase directly from the manufacturer.\n\n2. Amazon - One of the largest online retailers, Amazon, has a wide selection of Nvidia RTX 4090 graphics cards from different brands and sellers.\n\n3. Newegg - Another popular online marketplace for computer hardware, Newegg, has a variety of Nvidia RTX 4090 cards available for purchase.\n\n4. Best Buy - This electronics retailer has a decent selection of Nvidia RTX 4090 graphics cards available for purchase both online and in-store.\n\n5. Micro Center - A popular destination for computer enthusiasts, Micro Center, offers a range of Nvidia RTX 4090 cards at its physical stores.\n\n6. B&H Photo Video - This online retailer specializes in photography and video equipment but also offers a selection of Nvidia RTX 4090 graphics cards.\n\n7. Fry's Electronics - Another retail chain specializing in co

In [5]:
print(response["text"].strip())

1. Nvidia Online Store - The official website of Nvidia offers various RTX 4090 models for purchase directly from the manufacturer.

2. Amazon - One of the largest online retailers, Amazon, has a wide selection of Nvidia RTX 4090 graphics cards from different brands and sellers.

3. Newegg - Another popular online marketplace for computer hardware, Newegg, has a variety of Nvidia RTX 4090 cards available for purchase.

4. Best Buy - This electronics retailer has a decent selection of Nvidia RTX 4090 graphics cards available for purchase both online and in-store.

5. Micro Center - A popular destination for computer enthusiasts, Micro Center, offers a range of Nvidia RTX 4090 cards at its physical stores.

6. B&H Photo Video - This online retailer specializes in photography and video equipment but also offers a selection of Nvidia RTX 4090 graphics cards.

7. Fry's Electronics - Another retail chain specializing in computer hardware, Fry's Electronics, has a selection of Nvidia RTX 4090

In [6]:
prompt = PromptTemplate.from_template(
    input_variable=["product"],
    template="Where can I buy {product}?"
    )

# Setting verbose=True displays the detailed happenings in background
chain = LLMChain(llm=client, prompt=prompt, verbose=True)

response_1 = chain.invoke("Dragonfruit")
print(response_1["text"].strip())



> Entering new LLMChain chain...
Prompt after formatting:
Where can I buy Dragonfruit?

> Finished chain.
Dragonfruit can be purchased at most grocery stores and supermarkets, as well as specialty Asian markets and health food stores. It can also be purchased online from various retailers. Some places to buy dragonfruit include:

1. Whole Foods Market
2. Trader Joe's
3. Safeway
4. Walmart
5. Kroger
6. Asian markets (such as H Mart or 99 Ranch Market)
7. Health food stores (such as Sprouts or Natural Grocers)
8. Farmers' markets
9. Online retailers (such as Amazon or FreshDirect)


#### 5.1.2 LangChain Expression Language(LCEL)

**LCEL** is a declarative way to easily compose chains together. There are several benifits to writing chains in this manner as it allows for easy chaining of different components using **`|`** operator.

In [7]:
# Import necessary libraries
import json
from langchain.schema import BaseOutputParser
from langchain.prompts import PromptTemplate

class LLMOutputToJsonParser(BaseOutputParser):
    """ Parse the output of an LLM call to a JSON format """

    def parse(self, text: str) -> dict:
        countries = []
        lines = text.split("\n")

        for line in lines:
            parts = line.split('. ')
            if len(parts)>1:
                country_name = parts[1]
                countries.append(country_name)

        # Create a dict containing the countries 
        output_dict = {f"country{i+1}": country for i, country in enumerate(countries)}

        # Return after dict to a JSON string conversion
        return json.dumps(output_dict, indent=2)

In [8]:
model = OpenAI(openai_api_key=openai_key)  
prompt = PromptTemplate.from_template(
    template="Where countries import {product}?"
    )

# Chain the output parser, model and prompt using LCEL
chain = prompt | model | LLMOutputToJsonParser()
output = chain.invoke({"product": "rice"})
print(output)

{
  "country1": "Some of the largest importers of rice include China, Nigeria, Iran, Saudi Arabia, Indonesia, and the Philippines"
}


##### 5.1.2.1 Stream, Batch and Async support in LCEL

In [29]:
from langchain.prompts import ChatPromptTemplate

# Create a prompt template and chain the prompt with model
model = OpenAI(openai_api_key=openai_key, max_tokens=256)  
prompt = ChatPromptTemplate.from_template("Tell me about first ever successful {topic} in small paragraph")

chain = prompt | model

- __Streaming__ allows you to receive the output of a chian in chunks rather tha waiting for the entire output to be generated. This can be useful for processing large amounts of data or for real-time applications. To use streaming, __`.stream`__ method on a chain can be used.  

In [30]:
# Streaming
for s in chain.stream({"topic": "landing of Falcon Rocket by SpaceX"}):
    print(s, end="", flush="True")



On December 21, 2015, history was made as SpaceX successfully landed its Falcon 9 rocket for the first time. The rocket had just delivered 11 satellites into orbit for the telecommunications company, Orbcomm, and then made a controlled descent back to Earth. The landing took place at SpaceX's Landing Zone 1 at Cape Canaveral, Florida, where the rocket gracefully touched down on its landing legs. This was a monumental achievement as it marked the first time a rocket had ever been launched into orbit and then safely returned to land vertically. This successful landing marked a major milestone for SpaceX's goal of creating reusable rockets, reducing the cost of space travel, and ultimately making space exploration more accessible for all.

- __Batching__ allows you to process multiple inputs at once, which can be more efficient than processing them by one by one. To batch inputs, `.batch` method on a chain can be used.

In [81]:
# Batching
batch_prompts = [{"topic": "landing of Falcon Rocket by SpaceX"},
                 {"topic": "detection of ripple in fabric of space-time due to two blackholes merger"}]
                 
outputs = chain.batch(batch_prompts)

# Display the outputs
for i, result in enumerate(outputs):
    formatted_prompt = prompt.messages[0].prompt.template.format(topic=batch_prompts[i]['topic'])
    print(f"Prompt: {formatted_prompt}")
    print(f"Output: {result}\n")

Prompt: Tell me about first ever successful landing of Falcon Rocket by SpaceX in small paragraph
Output: 

On December 21, 2015, SpaceX made history by successfully landing the first stage of their Falcon 9 rocket back on Earth. This was a major milestone for the company, as it was the first time a rocket had ever been successfully landed after launching into orbit. The Falcon 9 had just delivered 11 satellites into space for the ORBCOMM-2 mission, and then made a controlled descent back to Earth. The rocket landed vertically on a landing pad at Cape Canaveral, Florida, marking a significant achievement for SpaceX and the future of space exploration. This successful landing demonstrated the potential for reusable rockets, which could greatly reduce the cost of space missions and make space travel more accessible. This accomplishment solidified SpaceX's position as a leader in the commercial space industry and paved the way for future successful landings and advancements in space techn

- __Async__ support allows you to invoke chains asynchronously, which can be useful for handling concurrent requests or for integrating with other async code. To invoke a chain asynchronously,

    - __`.ainvoke`__ method can be used for single invocations, 

    - __`.abatch`__ method can be used for batching and 
    
    - __`.astream`__ method for streaming.

In [83]:
# async
async def async_invoke():
    # Single invocation
    response = await chain.ainvoke({"topic": "landing of Falcon Rocket by SpaceX"})
    print(response)

    # Batching
    batch_prompts = [{"topic": "landing of Falcon Rocket by SpaceX"},
                 {"topic": "detection of ripple in fabric of space-time due to two blackholes merger"}]
                 
    responses = await chain.abatch(batch_prompts)

    # Display the outputs
    for i, result in enumerate(responses):
        formatted_prompt = prompt.messages[0].prompt.template.format(topic=batch_prompts[i]['topic'])
        print(f"Prompt: {formatted_prompt}")
        print(f"Output: {result}\n")

    # Streaming
    async for s in chain.astream({"topic": "landing of Falcon Rocket by SpaceX"}):
        print(s, end="", flush=True)

In [84]:
await async_invoke()



On December 21, 2015, SpaceX made history with the first ever successful landing of their Falcon 9 rocket. The rocket had launched from Cape Canaveral Air Force Station in Florida with 11 satellites for the telecommunications company Orbcomm. After the successful launch, the first stage of the rocket separated and began its descent back to Earth. Using a series of maneuvers and engine burns, the rocket was able to make a controlled landing on a landing pad at Cape Canaveral, marking the first time a rocket had ever landed intact after reaching orbit. This achievement was a major milestone for SpaceX and the space industry as a whole, as it proved the viability of reusable rockets and opened up possibilities for more cost-effective and sustainable space travel. 
Prompt: Tell me about first ever successful landing of Falcon Rocket by SpaceX in small paragraph
Output: 

On December 21, 2015, history was made when SpaceX successfully landed their Falcon 9 rocket at Cape Canaveral Air For

## 5.2 Various ways of Chaining 

In [90]:
# Import necessary libraries
from langchain_openai import OpenAI
from langchain.prompts import PromptTemplate, ChatPromptTemplate
from langchain.chains import LLMChain, SimpleSequentialChain

# Initiate OpenAI LLM model
model = OpenAI(temperature=0.7, max_tokens=128) 

### 5.2.1 Sequential Chains

A __sequential chain__ combines multiple chains where the output of one chain is the input of next chain. It runs a sequence of chains one after another. This is particularly useful when you want take the output from one call and use ir as the input to another.

<div align="center">
    <img src="https://av-eks-lekhak.s3.amazonaws.com/media/__sized__/article_images/5_83DdPYD-thumbnail_webp-600x300.webp" alt="LangChain Chains">
</div>

#### 5.2.1.1 Sequential Chain with Chain Interface

In [91]:
# First prompt
first_prompt = ChatPromptTemplate.from_template(
    "What is the best name to a describe a company that makes {product}?"
)

# Create first chain
chain_one = LLMChain(llm=model, prompt=first_prompt)

# Second prompt
second_prompt = PromptTemplate.from_template(
    template="What are the key strengths and unique selling points of {company}?"
)

# Create second chain
chain_two = LLMChain(llm=model, prompt=second_prompt)

In [92]:
# Create simple sequential chain
sequential_chain = SimpleSequentialChain(chains=[chain_one, chain_two],
                                         verbose=True)

# Invoke the sequential chain
response = sequential_chain.invoke("smartwatch")
print(response)



> Entering new SimpleSequentialChain chain...


"CyberTime" or "TechWrist" 


1. Cutting-edge Technology: Both CyberTime and TechWrist are equipped with the latest and most advanced technology, providing users with a seamless and efficient experience. This includes features such as AI assistants, voice control, and advanced tracking capabilities.

2. Sleek and Stylish Design: Both CyberTime and TechWrist are designed with a modern and sleek aesthetic, making them not only functional but also fashionable. This appeals to a wide range of users, from tech-savvy individuals to fashion-forward consumers.

3. Versatility: Both products offer a wide range of features and functions, making them versatile and suitable for various purposes.

> Finished chain.
{'input': 'smartwatch', 'output': '\n\n1. Cutting-edge Technology: Both CyberTime and TechWrist are equipped with the latest and most advanced technology, providing users with a seamless and efficient experience. This includes features su

In [95]:
print(response['output'].strip())

1. Cutting-edge Technology: Both CyberTime and TechWrist are equipped with the latest and most advanced technology, providing users with a seamless and efficient experience. This includes features such as AI assistants, voice control, and advanced tracking capabilities.

2. Sleek and Stylish Design: Both CyberTime and TechWrist are designed with a modern and sleek aesthetic, making them not only functional but also fashionable. This appeals to a wide range of users, from tech-savvy individuals to fashion-forward consumers.

3. Versatility: Both products offer a wide range of features and functions, making them versatile and suitable for various purposes.


#### 5.2.1.2 Sequential Chain with LCEL Method

In [ ]:
# First prompt
learning_prompt = PromptTemplate(
    input_variables=["activity"],
    template="I want to learn how to {activity}. Can you suggest how I can learn this step-by-step?"
)

# Second Prompt
time_prompt = PromptTemplate(
    input_variables=["learning_plan"],
    template="I only have one week. Can you create a concise plan to help me hit this goal: {learning_plan}."
)

In [ ]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model="gpt-4o-mini", api_key=openai_key)

# Complete the sequential chain with LCEL
seq_chain = ({"activities": destination_prompt | llm | StrOutputParser()}
    | activities_prompt    
    | llm    
    | StrOutputParser())

# Call the chain
print(seq_chain.invoke({"activity": "play the harmonica"}))

### 5.2.2 Router Chains

A __router chain__ is used for complicated tasks. If we have multiple subchains, each of which is specialized for a particular type of input, we could have a router chain that decides which subchain to pass the input to.

#### 5.2.2.1 Router Chain Components

1. **Router Chain**: Manages and directs the flow to appropriate destination chains based on input or conditions.

2. **Destination Chains**: Specific chains that handle different tasks or functionalities.

3. **Default Chain**: A fallback option used when no specific destination chain is selected.

</br>
<div align="center">
    <img src="https://av-eks-lekhak.s3.amazonaws.com/media/__sized__/article_images/9_yeEjVhG-thumbnail_webp-600x300.webp" alt="LangChain Chains">
</div>

In [99]:
# Import necessary libraries
from langchain_openai import OpenAI
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain
from langchain.chains.router import MultiPromptChain
from langchain.chains.router.llm_router import LLMRouterChain, RouterOutputParser

# Initiate OpenAI LLM model
model = OpenAI(temperature=0.7, max_tokens=128) 

In [97]:
# Creating a Role-play Prompting
physics_template = """You are a very smart physics professor. \
You are great at answering questions about physics in a concise\
and easy to understand manner. \
When you don't know the answer to a question you admit\
that you don't know.

Here is a question:
{input}"""


math_template = """You are a very good mathematician. \
You are great at answering math questions. \
You are so good because you are able to break down \
hard problems into their component parts,
answer the component parts, and then put them together\
to answer the broader question.

Here is a question:
{input}"""


history_template = """You are a very good historian. \
You have an excellent knowledge of and understanding of people,\
events and contexts from a range of historical periods. \
You have the ability to think, reflect, debate, discuss and \
evaluate the past. You have a respect for historical evidence\
and the ability to make use of it to support your explanations \
and judgements.

Here is a question:
{input}"""

In [98]:
# Defining the prompt templates
prompt_infos = [
    {
        "name": "physics",
        "description": "Good for answering questions about physics",
        "prompt_template": physics_template
    },
    {
        "name": "math",
        "description": "Good for answering math questions",
        "prompt_template": math_template
    },
    {
        "name": "history",
        "description": "Good for answering questions about history",
        "prompt_template": history_template
    }
]

__Destination Chain__

In [100]:
# Create a destination chain
destination_chains = {}

for info in prompt_infos:
    name = info["name"]
    prompt_template = info["prompt_template"]
    prompt = ChatPromptTemplate.from_template(template=prompt_template)
    chain = LLMChain(llm=model, prompt=prompt)
    destination_chains[name] = chain

destinations = [f"{p['name']}: {p['description']}" for p in prompt_infos]
destinations_str = "\n".join(destinations)

__Multi-prompt Router Template__

In [101]:
MULTI_PROMPT_ROUTER_TEMPLATE = """Given a raw text input to a \
language model select the model prompt best suited for the input. \
You will be given the names of the available prompts and a \
description of what the prompt is best suited for. \
You may also revise the original input if you think that revising\
it will ultimately lead to a better response from the language model.

<< FORMATTING >>
Return a markdown code snippet with a JSON object formatted to look like:
```json
{{{{
    "destination": string \ name of the prompt to use or "DEFAULT"
    "next_inputs": string \ a potentially modified version of the original input
}}}}
```

REMEMBER: "destination" MUST be one of the candidate prompt \
names specified below OR it can be "DEFAULT" if the input is not\
well suited for any of the candidate prompts.
REMEMBER: "next_inputs" can just be the original input \
if you don't think any modifications are needed.

<< CANDIDATE PROMPTS >>
{destinations}

<< INPUT >>
{{input}}

<< OUTPUT (remember to include the ```json)>> \
"""

__Default Chain__ 

In [102]:
# Creating a router prompt and chaining
default_prompt = ChatPromptTemplate.from_template("{input}")
default_chain = LLMChain(llm=model, prompt=default_prompt)

__Router Template__

In [104]:
# Creating a router template and chain
router_template = MULTI_PROMPT_ROUTER_TEMPLATE.format(destinations=destinations_str)

router_prompt = PromptTemplate(
    template=router_template,
    input_variables=["input"],
    output_parser=RouterOutputParser()
)

router_chain = LLMRouterChain.from_llm(llm=model, prompt=router_prompt)

__Chaining Everything Together__

In [114]:
# Chaining Everything Together
chain = MultiPromptChain(
    router_chain=router_chain,
    destination_chains=destination_chains,
    default_chain=default_chain,
    verbose=True
)

# Invoke the chain for physics
response = chain.invoke("What is a blackhole?")
response



> Entering new MultiPromptChain chain...
physics: {'input': 'What is a blackhole?'}
> Finished chain.


{'input': 'What is a blackhole?',
 'text': '\n\nA black hole is a region of space where the gravitational pull is so strong that nothing, including light, can escape from it. It is formed when a massive star collapses under its own gravity after running out of nuclear fuel. The gravitational pull of a black hole is so strong because all of the mass of the star is compressed into a very small space, creating a massive curvature in space-time. This extreme curvature is what gives a black hole its strong gravitational pull.'}

In [117]:
print(response['text'].strip())

A black hole is a region of space where the gravitational pull is so strong that nothing, including light, can escape from it. It is formed when a massive star collapses under its own gravity after running out of nuclear fuel. The gravitational pull of a black hole is so strong because all of the mass of the star is compressed into a very small space, creating a massive curvature in space-time. This extreme curvature is what gives a black hole its strong gravitational pull.


In [128]:
# Invoke the chain for history related
history_response = chain.invoke("When did first World War happen?")



> Entering new MultiPromptChain chain...
history: {'input': 'When did first World War happen?'}
> Finished chain.


In [129]:
print(history_response['text'].strip())

The First World War, also known as the Great War, took place from July 28, 1914 to November 11, 1918.


In [132]:
# Invoke for both phyiscs, math and  history related
combined_prompt = """You are an expert in physics, math, and history. \
Your task is to provide a detailed response that integrates these disciplines. Consider the following question:

{question}

Make sure to cover:
1. The historical context and significance of the event.
2. The scientific principles and physical theories related to the event.
3. The mathematical modeling and calculations influenced by the event.

Provide a comprehensive answer that connects these aspects seamlessly."""

question = "How did the development of radar during World War II influence mathematical modeling and physical theories?"

combined_response = chain.invoke(combined_prompt.format(question=question))



> Entering new MultiPromptChain chain...
None: {'input': 'You are an expert in physics, math, and history. Your task is to provide a detailed response that integrates these disciplines. Consider the following question: How did the development of radar during World War II influence mathematical modeling and physical theories? Make sure to cover: 1. The historical context and significance of the event. 2. The scientific principles and physical theories related to the event. 3. The mathematical modeling and calculations influenced by the event. Provide a comprehensive answer that connects these aspects seamlessly.'}
> Finished chain.


In [133]:
print(combined_response['text'].strip())

The development of radar during World War II had a significant impact on both mathematical modeling and physical theories. It was a pivotal moment in history that not only revolutionized warfare, but also advanced our understanding of electromagnetism and paved the way for modern radar technology.

Firstly, let's provide some historical context. Prior to World War II, scientists had already established the principles of electromagnetism, including the propagation of electromagnetic waves and the concept of resonance. However, it was not until the war that these principles were put into practice and perfected in the form of radar technology. Radar, or Radio Detection And Ranging, was initially developed
